## Usando o detectores_json_k_dobras no Colab

Este notebook executa um experimento completo, com validação cruzada em k-dobras para comparar diferentes redes profundas de detecção de objetos.


## Preparando o código

* Se baixou o projeto pelo zip, depois de descompactar, renomeie a pasta de detectores_json_k_dobras_master para detectores_json_k_dobras

* Crie a pasta checkpoints dentro de detectores_json_k_dobras e coloque lá os arquivos .pth das redes que você pretende usar (leia no README.md do detectores_json_k_dobras a seção "Escolhendo as arquiteturas a serem testadas"). 

* Copie a pasta com todos os arquivos do detectores_json_k_dobras (incluindo os checkpoints) para sua conta no Google Drive (+ NOVO -> Upload de Pasta.

* Rode a linha abaixo para que o Colab possa começar a acessar os seus dados no Drive (o Drive vai pedir que você autorize o acesso e dependendo do tipo de conta vai também gerar um código longo para você copiar no Colab, leia e siga as orientações).

* Se der pau, tente o menu "Runtime->Restart Runtime"



In [ ]:
# Liga o Colab à sua conta no Drive
from google.colab import drive
drive.mount('/content/drive')

# Entra na pasta do detectores_json_k_dobras que você subiu
# Se subiu em outro lugar, ajuste o comando abaixo
%cd /content/drive/MyDrive/detectores_json_k_dobras

# Lista o conteúdo da pasta para conferir se está tudo OK
!ls 

## Instalando os pacotes necessários

Na primeira vez que instala pode demorar uns 20 minutos (ou mais) para instalar tudo. Não se desespere !!! Para rodar o experimento geralmente demora MUITO MAIS :-) 


#### Instala o pytorch e o mmcv

In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full==1.3.5

# Instalar biblioteca que o gerador de dobras precisa
!pip install funcy



### Ajusta o mmdetection

Dentro do detectores_json_k_dobras tem uma versão do mmdetection que não é a última, mas que estava funcionando para nós (se quiser tentar a última descomente os comandos rm e git abaixo)

PS: Se pedir para "Restart Runtime" depois que instalar a versão mais antiga do Pillow, pode clicar sem medo.

In [ ]:
# Se quiser instalar a última versão do mmdetection descomente
# as linhas abaixo (por usa conta e risco)
# !rm -rf mmdetection
# !git clone https://github.com/open-mmlab/mmdetection.git

# Entra na pasta do mmdetection e ajusta as coisas
%cd mmdetection
!pip install -e .


# Se der algum pau relacionado com o Pillow
# tente descomentar a linha de baixo para
# instalar uma versão mais antiga.
# !pip install Pillow==7.0.0



## Testa a instalação

In [ ]:
# Verificando instalação do Pytorch 
import torch, torchvision
print("Versão do Torch instalada: ",torch.__version__)
print("Achou o CUDA para usar a GPU: ", torch.cuda.is_available())

# Verificando instalação do mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print("Versão do CUDA usada no MMCV: ", get_compiling_cuda_version())
print("Versão do compilador usado no MMCV: ",get_compiler_version())

# Verificando instalação do MMDetection
import mmdet
print("MMDET version: ",mmdet.__version__)



## Preparando os dados 

Use o Roboflow para anotar e gerar um arquivo compactado (.zip) com as imagens e anotações. Use a opção para deixar tudo junto em um único conjunto de treinamento. No canal do youtube do Prof. Pistori tem um vídeo sobre o Roboflow. 

Coloque o arquivo gerado no roboflow dentro da pasta do detectores_json_k_dobras no seu Google Drive

Troque dentro do arquivo experimento.py, nas linhas 10 e 11, o nome da classe que você usou para anotar suas imagens (no lugar de **eucaliptos**) e o número de dobras, caso não vá usar 5 dobras

***NÃO RODE O CÓDIGO ABAIXO SE QUISER APENAS USAR O EXEMPLO QUE JÁ VEM PRONTO***


In [ ]:
# Vai para a pasta onde está o código
%cd /content/drive/MyDrive/detectores_json_k_dobras

# IMPORTANTE !!! Troque o nome "suas_imagens_roboflow.zip" 
# pelo nome do arquivo .zip que você criou no roboflow
!mv bois*.zip ./dataset/all
%cd ./dataset/all/  
!rm -rf train
!unzip bois*.zip


# Gera as dobras da validação cruzada

In [ ]:
# Altere abaixo o número de dobras (-folds) e o percentual de 
# validação (-valperc) se necessário. 
# Os resultados desta etapa ficarão na pasta ./dataset/filesJSON 
%cd /content/drive/MyDrive/detectores_json_k_dobras/utils
!python geraDobras.py -folds=5 -valperc=0.3  # Gera as dobras para a validação cruzada 


## Rodando o experimento

O comando abaixo é que roda o experimento (que depende de GPU).IMPORTANTE: Veja no README do projeto, seção "Escolhendo as arquiteturas a serem testadas
", como baixar os checkpoints (arquivo .pth) das redes que você irá utilizar. Estes checkpoints também precisam ser colocados no drive, dentro da pasta detectores_json_k_dobras/checkpoints/ 

In [ ]:
# Muda novamente para a pasta correta
%cd /content/drive/MyDrive/detectores_json_k_dobras

# Roda o experimento
!python experimento.py

## Gerando os gráficos com o R

Vai instalar alguns pacotes adicionais e rodar o código em R (demora alguns minutos)

In [ ]:
%cd /content/drive/MyDrive/detectores_json_k_dobras
!Rscript --version
!Rscript install_R_packages.R

In [ ]:
%cd /content/drive/MyDrive/detectores_json_k_dobras
!Rscript graficos.R

## Mostra alguns dos resultados

Os resultados consolidados ficam dentro da pasta /content/drive/MyDrive/detectores_json_k_dobras/dataset . Para resultados mais detalhados e imagens com as detecções de objetos entre em pastas como esta aqui: /content/drive/MyDrive/detectores_json_k_dobras/dataset/prediction_faster 

In [ ]:
from IPython.display import Image, display

%cd /content/drive/MyDrive/detectores_json_k_dobras/dataset
!cat statistics.txt
display(Image('boxplot.png'))
display(Image('history.png'))
display(Image('counting.png'))
display(Image('histogram.png'))



# Truque para evitar que o Colab desconecte 

Depois de muito tempo sem interação o Colab pode desconectar. Use F12 para abrir a janela de código do Google Chrome. Clique em Console (Veja uma aba na parte de baixo desta janela de código). Copie o código abaixo para a linha de comando do console e tecle ENTER. Use F12 novamente para fechar a janela de código.

-----

function ClickConnect(){
  console.log("Connnect Clicked - Start"); 
  document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
  console.log("Connnect Clicked - End"); 
};
setInterval(ClickConnect, 60000)

-----

Link explicando este truque: https://www.ti-enxame.com/pt/python/como-impedir-que-o-google-colab-se-desconecte/810821538/
